In [1]:
from gym import Env
from gym.spaces import Discrete, Tuple
from gym.utils.env_checker import check_env

import numpy as np
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

from py4j.java_gateway import JavaGateway

In [2]:
# Conecta no servidor de treinamento

gateway = JavaGateway()
minitruco_java = gateway.entry_point 

In [3]:
###### Env customizado do gym que encapsupla o servidor

class MinitrucoEnv(Env):
    def __init__(self):
        # Ações que podemos tomar: jogar a carta 0, a carta 1 ou a carta 2
        self.action_space = Discrete(3)
        
        # Estados possíveis
        # - posições: 1=inferior, 2=direita, 3=superior, 4=esquerda    
        # - equipes: 1=posições 1 e 3; 2=posições 2 e 4
        # - cartas: podem valer -1 (null), 0 (fechada) ou um valor de 1
        #           a 14, conforme o valor relativo delas (cartas normais de 1
        #           a 10, manilhas de 11 a 14)
        # - rodadas: 1 a 3
        # - resultado da rodada: a equipe que venceu (1 ou 2), 3 para empate ou -1 para rodada não conlcuída
        # - booleanos (ex.: podeFechada) são 0 ou 1
        # TBD posJogadorPedindoAumento (acho que não zera depois do aumento)
        # TBD tento mineiro (talvez só varie as recompensas, mas é preciso especificar)
        # TBD baralho limpo (provavelmente só vamos excluir o range 1-4)
        self.observation_space = Tuple((
            Discrete(4, start=1),    # posJogador
            Discrete(2),             # baralhoSujo
            Discrete(2),             # podeFechada
            Discrete(3, start=1),    # numRodadaAtual
            Discrete(5, start=-1),   # resultadoRodada1
            Discrete(5, start=-1),   # resultadoRodada2
            Discrete(12, start=1),   # valorMao
            Discrete(13, start=0),   # valorProximaAposta
            Discrete(5, start=0),    # posJogadorPedindoAumento
            Discrete(4, start=1),    # posJogadorQueAbriuRodada
            Discrete(24, start=0),   # pontosEquipe1
            Discrete(24, start=0),   # pontosEquipe2
            Discrete(16, start=-1),  # cartaJogadaRodada1Pos1
            Discrete(16, start=-1),  # cartaJogadaRodada1Pos2
            Discrete(16, start=-1),  # cartaJogadaRodada1Pos3
            Discrete(16, start=-1),  # cartaJogadaRodada1Pos4
            Discrete(16, start=-1),  # cartaJogadaRodada2Pos1
            Discrete(16, start=-1),  # cartaJogadaRodada2Pos2
            Discrete(16, start=-1),  # cartaJogadaRodada2Pos3
            Discrete(16, start=-1),  # cartaJogadaRodada2Pos4
            Discrete(16, start=-1),  # cartaJogadaRodada3Pos1
            Discrete(16, start=-1),  # cartaJogadaRodada3Pos2
            Discrete(16, start=-1),  # cartaJogadaRodada3Pos3
            Discrete(16, start=-1),  # cartaJogadaRodada3Pos4
            Discrete(16, start=-1),  # carta1Jogador
            Discrete(16, start=-1),  # carta2Jogador
            Discrete(16, start=-1)   # carta3Jogador
        ))
        
        self.episodio = None
        self.state = None
        
    def step(self, action):
        last_observation = self.state
        cartaJogada = last_observation[24 + action]
        if cartaJogada == -1:
            # Jogada inválida
            return last_observation, 0, False, False, {}
        
        self.episodio.executa(action)
        estado_str = self.episodio.estado()
        if estado_str == "EQUIPE 1 VENCEU":
            return None, 1.0, True, False, {}
        if estado_str == "EQUIPE 2 VENCEU":
            return None, -1.0, True, False, {}
                
        self.state = tuple(map(int, estado_str.split()))
        
        pontosEquipe1 = self.state[10]
        pontosEquipe2 = self.state[11]
        
        reward = 0.0
        terminated = False
        
        return self.state, reward, terminated, False, {}

    def render(self):
        pass
    
    def reset(self, seed=None, options=None):
        if self.episodio is not None:
            self.episodio.finaliza()
            
        super().reset(seed=seed) # required by check_env
        
        self.episodio = minitruco_java.novoEpisodio()
        self.state = tuple(map(int, self.episodio.estado().split()))
        
        return (self.state, {})
    

In [4]:
# Inicializa e confere se está tudo certo
    
env = MinitrucoEnv()
check_env(env)
env.episodio.finaliza() # Necessário porque check_env não termina o jogo

In [5]:
# Roda alguns episódios com política "jogue uma carta aleatoriamente"

import time

episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    terminated = False
    score = 0 
    start = time.time()
    
    while not terminated:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, terminated, truncated, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{} Time:{}s'.format(episode, score, time.time() - start))

Episode:1 Score:-1.0 Time:0.3589801788330078s
Episode:2 Score:-1.0 Time:0.40641307830810547s
Episode:3 Score:-1.0 Time:0.5237131118774414s
Episode:4 Score:-1.0 Time:0.41701364517211914s
Episode:5 Score:-1.0 Time:0.48198413848876953s
Episode:6 Score:1.0 Time:0.45208120346069336s
Episode:7 Score:-1.0 Time:0.45982813835144043s
Episode:8 Score:1.0 Time:0.4103529453277588s
Episode:9 Score:-1.0 Time:0.46395206451416016s
Episode:10 Score:1.0 Time:0.5101392269134521s
